In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = 2048
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
batch_size = 64
epoch = 100

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
FGSM = onp.load('./cifar-targeted-fgsm-x.npy')
I_FGSM = onp.load('./cifar-targeted-ifgsm-x.npy')

In [6]:
y_test = onp.load("./cifar-targeted-clean-y.npy")

In [7]:
x_test = onp.load("./cifar-targeted-clean-x.npy")

In [8]:
layers = tf.keras.layers

In [9]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64,  (3, 3), activation='relu', padding='same')(img_input)
x = layers.Conv2D(64,  (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPool2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(256, activation='relu')(x)
out = layers.Dense(10)(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [18]:
def scheduler(epoch, lr):
    if epoch < 5:
        return 1e-4
    elif epoch < 55:
        return 1e-2
    elif epoch < 105:
        return 1e-2
    else:
        return 1e-4

In [19]:
# model.compile(optimizer=tf.keras.optimizers.SGD(),
#               loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

In [20]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.MSE,
              metrics=['accuracy'])

In [21]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [22]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)        

In [23]:
x_train_all, x_test_all = x_train_all.reshape(-1, 32, 32, 3), x_test_all.reshape(-1, 32, 32, 3)

In [25]:
# model = tf.keras.models.load_model('./vgg-19-cifar10-lr-schedule.h5')
model.fit(x_train_all, y_train_all, batch_size=batch_size, epochs=150, validation_split=0.1, callbacks=[callback])

Epoch 1/150
704/704 [==============================] - 10s 15ms/step - loss: 0.0521 - accuracy: 0.6066 - val_loss: 0.0533 - val_accuracy: 0.5910
Epoch 2/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0521 - accuracy: 0.6058 - val_loss: 0.0533 - val_accuracy: 0.5916
Epoch 3/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0521 - accuracy: 0.6062 - val_loss: 0.0533 - val_accuracy: 0.5908
Epoch 4/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0521 - accuracy: 0.6070 - val_loss: 0.0533 - val_accuracy: 0.5908
Epoch 5/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0521 - accuracy: 0.6064 - val_loss: 0.0532 - val_accuracy: 0.5912
Epoch 6/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0539 - accuracy: 0.5893 - val_loss: 0.0677 - val_accuracy: 0.4620
Epoch 7/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0537 - accuracy: 0.5909 - val_loss: 0.0641 - val_ac

704/704 [==============================] - 10s 14ms/step - loss: 0.0388 - accuracy: 0.7229 - val_loss: 0.0666 - val_accuracy: 0.5008
Epoch 58/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0385 - accuracy: 0.7259 - val_loss: 0.0584 - val_accuracy: 0.5678
Epoch 59/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0381 - accuracy: 0.7267 - val_loss: 0.0528 - val_accuracy: 0.6120
Epoch 60/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0378 - accuracy: 0.7302 - val_loss: 0.0486 - val_accuracy: 0.6348
Epoch 61/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0375 - accuracy: 0.7339 - val_loss: 0.0469 - val_accuracy: 0.6500
Epoch 62/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0372 - accuracy: 0.7359 - val_loss: 0.0499 - val_accuracy: 0.6274
Epoch 63/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0369 - accuracy: 0.7392 - val_loss: 0.0562 - val_accuracy

704/704 [==============================] - 10s 14ms/step - loss: 0.0182 - accuracy: 0.8914 - val_loss: 0.0437 - val_accuracy: 0.6832
Epoch 114/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0181 - accuracy: 0.8917 - val_loss: 0.0437 - val_accuracy: 0.6830
Epoch 115/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0181 - accuracy: 0.8918 - val_loss: 0.0437 - val_accuracy: 0.6842
Epoch 116/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0181 - accuracy: 0.8922 - val_loss: 0.0438 - val_accuracy: 0.6846
Epoch 117/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0180 - accuracy: 0.8923 - val_loss: 0.0438 - val_accuracy: 0.6832
Epoch 118/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0180 - accuracy: 0.8924 - val_loss: 0.0438 - val_accuracy: 0.6836
Epoch 119/150
704/704 [==============================] - 10s 14ms/step - loss: 0.0180 - accuracy: 0.8930 - val_loss: 0.0438 - val_ac

In [19]:
tmp = onp.load('./cifar-untargeted-variance-fgsm.npy')

In [20]:
clean_x = onp.load('./cifar-clean-x.npy')
clean_y = onp.load('./cifar-clean-y.npy')

In [21]:
model.evaluate(clean_x, clean_y)

4/4 [==============================] - 1s 296ms/step - loss: 1.6793 - accuracy: 0.8047


[1.679345726966858, 0.8046875]

In [22]:
tmp = onp.load('./cifar-untargeted-simple-fgsm.npy')

In [23]:
model.evaluate(tmp, clean_y)

4/4 [==============================] - 0s 8ms/step - loss: 4.0191 - accuracy: 0.5391


[4.019050121307373, 0.5390625]

In [21]:
model.evaluate(tmp, clean_y)

4/4 [==============================] - 0s 14ms/step - loss: 1.9319 - accuracy: 0.7344


[1.9318859577178955, 0.734375]

In [20]:
sum(onp.argmax(model(tmp), axis=1)[onp.argmax(model(x_test), axis=1) == onp.argmax(y_test, axis=1)]==0) /\
onp.argmax(model(x_test), axis=1)[onp.argmax(model(x_test), axis=1) == onp.argmax(y_test, axis=1)].shape

array([0.05399792])